In [ ]:
### APO Minimization and Equilibration test ###

In [2]:

from SBLMDCOVDOCK import SBLSettings

settings = SBLSettings.GROMACS_Settings()

import pandas as pd
import os

import gmxapi as gmx
import subprocess

In [3]:
# collect APO structures 

# AMPC - https://journals.asm.org/doi/10.1128/AAC.02073-20
ampc = "6T3D"
# KPC-2 - BLDB: http://dx.doi.org/10.1021/ACS.JMEDCHEM.7B00158
kpc2 = "5UL8"
# OXA-10 - BLDB: https://www.pnas.org/doi/full/10.1073/pnas.241442898
oxa10 = "1K55"

structures = pd.DataFrame({"PDBID": [ampc, kpc2, oxa10], 
                           "Name": ["AmpC", "KPC2", "OXA10"]})



In [6]:
# Create Box, Solvate, add Ions
for pdbcode in structures.PDBID:
    break # tested working 
    print(pdbcode)  
    # Input path
    input_path = os.path.join(settings.structures_output, "APO_" + pdbcode + ".gro")
    
    # Output paths
    box_output_path = input_path.replace(".gro", "_box.pdb")

    # Define the editconf command
    editconf_command = ["gmx", "editconf", "-f", input_path, "-o", box_output_path, "-c", "-d", "1.0", "-bt", "cubic"]

    # Run the editconf command
    subprocess.run(editconf_command, check=True) 
    # subprocess.run(editconf_command.replace(".pdb",".gro"), check=True) 


    solvated_output_path = input_path.replace(".gro", "_solv.gro")

    topo_path = input_path.replace(".gro", ".top")

    # Define the solvate command
    solvate_command = ["gmx", "solvate", "-cp", box_output_path, "-cs", "spc216.gro", "-p", topo_path, "-o", solvated_output_path]

    # Run the solvate command
    subprocess.run(solvate_command, check=True)

    ion_MDP = os.path.join(settings.config, "ions.mdp")
    # Generate tpr for genion
    grompp_command = ["gmx", "grompp", "-f", ion_MDP, "-c", solvated_output_path, "-p", topo_path, "-o", solvated_output_path.replace(".gro", ".tpr")]

    # Run the grompp command
    subprocess.run(grompp_command, check=True)

    ionised_output_path = solvated_output_path.replace(".gro", "_ion.gro")

    # Define the genion command
    genion_command = ["gmx", "genion", "-s", solvated_output_path.replace(".gro", ".tpr"), "-o", ionised_output_path, "-p", topo_path, "-pname", "NA", "-nname", "CL", "-neutral", "-conc", "0.15"]

    # Run the genion command
    subprocess.run(genion_command, input='SOL\n', check=True, text=True)

    


In [11]:
# Perform minimisation

for pdbcode in structures.PDBID:
    # prep minimisation
    min_mdp = os.path.join(settings.config, "minim.mdp")
    input_path = os.path.join(settings.structures_output, "APO_" + pdbcode + "_solv_ion.gro")
    topo_path = os.path.join(settings.structures_output, "APO_" + pdbcode+ ".top")
    tpr_path = os.path.join(settings.structures_output, "APO_" + pdbcode + "_min.tpr")
    grompp_command = ["gmx", "grompp", "-f", min_mdp, "-c", input_path, "-p", topo_path, "-o", tpr_path]

    subprocess.run(grompp_command, check=True)

    # Run minimization
    min_command = ["gmx", "mdrun", "-v", "-deffnm", tpr_path.replace(".tpr","")]
    subprocess.run(min_command, check=True)

    # # Get energy
    # energy_command = ["gmx", "energy", "-f", tpr_path.replace(".tpr",".edr"), "-o", tpr_path.replace(".tpr",".xvg")]
    # subprocess.run(energy_command, check=True)
    

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_2/Project/SBL_MD_CovDock
Command line:
  gmx grompp -f config/minim.mdp -c prod_structures/APO/APO_6T3D_solv_ion.gro -p prod_structures/APO/APO_6T3D.top -o prod_structures/APO/APO_6T3D_min.tpr

Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file config/minim.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group rest is 138753.00
The integrator does not provide a ensemble temperature, there is no system ensemble temperature

There was 1 NOTE

Back Off! I just backed up prod_structures/APO/APO_6T3D_min.tpr to prod_structures/APO/#APO_6T3D_min.tpr.6#

GROMACS remi

Setting the LD random seed to -537964681

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

Excluding 2 bonded neighbours molecule type 'SOL'

Excluding 1 bonded neighbours molecule type 'NA'

Excluding 1 bonded neighbours molecule type 'CL'
Analysing residue names:
There are:   358    Protein residues
There are: 20272      Water residues
There are:   127        Ion residues
Analysing Protein...

The largest distance between excluded atoms is 0.417 nm between atom 5203 and 5211
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 80x80x80, spacing 0.110 0.110 0.110

Estimate for the relative computational load of the PME mesh part: 0.23

This run will generate roughly 5 Mb of data


Using 10 MPI threads

Back Off! I just backed up prod_structures/APO/APO_6T3D_min.trr to prod_structures/APO/#APO_6T3D_min.trr.2#

Back Off! I just backed up prod_structures/APO/APO_6T3D_min.edr to prod_structures/APO/#APO_6T3D_min.edr.2#

Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+03
   Number of steps    =        50000
Step=    0, Dmax= 1.0e-02 nm, Epot= -2.52123e+05 Fmax= 1.32918e+07, atom= 175
Step=    1, Dmax= 1.0e-02 nm, Epot= -3.44500e+05 Fmax= 1.55263e+06, atom= 175
Step=    2, Dmax= 1.2e-02 nm, Epot= -3.87852e+05 Fmax= 3.28679e+05, atom= 66489
Step=    3, Dmax= 1.4e-02 nm, Epot= -4.70960e+05 Fmax= 8.46384e+04, atom= 66489
Step=    4, Dmax= 1.7e-02 nm, Epot= -5.98051e+05 Fmax= 2.26005e+04, atom= 66518
Step=    5, Dmax= 2.1e-02 nm, Epot= -7.02945e+05 Fmax= 8.12312e+03, atom= 1754
Step=    6, Dmax= 2.5e-02 nm, Epot= -7.63826e+05 Fmax= 1.54118e+04, atom= 4863
Step=    7, Dmax= 3.0e-02 nm, Epot= -7.79359e+05 Fmax= 2.27252e+04, atom= 5562
Step=    8, Dmax= 3.6e-02 nm, Epot

KeyboardInterrupt: 

In [ ]:
# Check Minimization 
for pdbcode in structures.PDBID:


In [ ]:
#Perform equilibration of the apo protein
for pdbcode in structures.PDBID:
    